# 第3章: 正規表現 

Wikipediaの記事を以下のフォーマットで書き出したファイル[jawiki-country.json.gz](http://www.cl.ecei.tohoku.ac.jp/nlp100/data/jawiki-country.json.gz)がある．

- 1行に1記事の情報がJSON形式で格納される
- 各行には記事名が"title"キーに，記事本文が"text"キーの辞書オブジェクトに格納され，そのオブジェクトがJSON形式で書き出される
- ファイル全体はgzipで圧縮される

以下の処理を行うプログラムを作成せよ．

## 参考（pythonでの正規表現の扱い：reモジュール）

詳しくは、[http://docs.python.jp/3/library/re.html#module-re](http://docs.python.jp/3/library/re.html#module-re)を参照

## 20. JSONデータの読み込み

Wikipedia記事のJSONファイルを読み込み，「イギリス」に関する記事本文を表示せよ．問題21-29では，ここで抽出した記事本文に対して実行せよ．

In [ ]:
%%bash
wget http://www.cl.ecei.tohoku.ac.jp/nlp100/data/jawiki-country.json.gz

In [ ]:
%%bash
gzip -d jawiki-country.json.gz

In [ ]:
%%bash
head jawiki-country.json

In [ ]:
import json

with open("jawiki-country.json") as datafile:
    for line in datafile:
        json_data = json.loads(line)
        #print(json_data["title"])
        if json_data["title"] == "イギリス":
            #print("hoge!!!")
            print(json_data["text"])

#### qiitaの答え
```
# coding: utf-8
import gzip
import json
fname = 'jawiki-country.json.gz'

with gzip.open(fname, 'rt') as data_file:
    for line in data_file:
        data_json = json.loads(line)
        if data_json['title'] == 'イギリス':
            print(data_json['text'])
            break
```

## 21. カテゴリ名を含む行を抽出

記事中でカテゴリ名を宣言している行を抽出せよ．

In [ ]:
import json

def getEnglandJson():
    with open("jawiki-country.json") as datafile:
        for line in datafile:
            json_data = json.loads(line)
            if json_data["title"] == "イギリス":
                return json_data["text"]

print(getEnglandJson())

In [ ]:
row_list21 = getEnglandJson().split("\n")
print(len(row_list21))
for row in row_list21:
    if row[0:11] == "[[Category:":
        print(row)

#### qiitaの答え
```
# coding: utf-8
import gzip
import json
import re
fname = 'jawiki-country.json.gz'


def extract_UK():
    '''イギリスに関する記事本文を取得

    戻り値：
    イギリスの記事本文
    '''

    with gzip.open(fname, 'rt') as data_file:
        for line in data_file:
            data_json = json.loads(line)
            if data_json['title'] == 'イギリス':
                return data_json['text']

    raise ValueError('イギリスの記事が見つからない')


# 正規表現のコンパイル
pattern = re.compile(r'''
    ^   # 行頭
    (   # キャプチャ対象のグループ開始
    .*  # 任意の文字0文字以上
    \[\[Category:
    .*  # 任意の文字0文字以上
    \]\]
    .*  # 任意の文字0文字以上
    )   # グループ終了
    $   # 行末
    ''', re.MULTILINE + re.VERBOSE)

# 抽出
result = pattern.findall(extract_UK())

# 結果表示
for line in result:
    print(line)
```

## 22. カテゴリ名の抽出

記事のカテゴリ名を（行単位ではなく名前で）抽出せよ．

In [ ]:
import re

pattern22 = re.compile(r'^.*\[\[Category:(.*)\]\].*$', re.MULTILINE + re.VERBOSE)
result22 = pattern22.findall(getEnglandJson())

for line in result22:
    print(line)

#### qiitaの答え
```
# 正規表現のコンパイル
pattern = re.compile(r'''
    ^       # 行頭
    .*      # 任意の文字0文字以上
    \[\[Category:
    (       # キャプチャ対象のグループ開始
    .*?     # 任意の文字0文字以上、非貪欲マッチ（貪欲にすると後半の'|'で始まる装飾を巻き込んでしまう）
    )       # グループ終了
    (?:     # キャプチャ対象外のグループ開始
    \|.*    # '|'に続く0文字以上
    )?      # グループ終了、0か1回の出現
    \]\]
    .*      # 任意の文字0文字以上
    $       # 行末
    ''', re.MULTILINE + re.VERBOSE)

# 抽出
result = pattern.findall(extract_UK())

# 結果表示
for line in result:
    print(line)
```

###### または

```
# 正規表現のコンパイル その2
pattern = re.compile(r'''
    ^       # 行頭
    .*      # 任意の文字0文字以上
    \[\[Category:
    (.*?)   # キャプチャ対象、任意の文字0文字以上、非貪欲マッチ
    (?:\]\]|\|) # キャプチャ対象外、']]'または'|'
    .*      # 任意の文字0文字以上
    $       # 行末
    ''', re.MULTILINE + re.VERBOSE)
```

## 23. セクション構造

記事中に含まれるセクション名とそのレベル（例えば"== セクション名 =="なら1）を表示せよ．

In [ ]:
import json
import re

pattern23 = re.compile(r'^(==+.+==+).*$', re.MULTILINE + re.VERBOSE)
result23 = pattern23.findall(getEnglandJson())

for line in result23:
    temp = line[1:]
    level = 0
    while temp[0] == "=":
        level += 1
        temp = temp[1:]
    print(line, level)

#### qiitaの答え
```
# 正規表現のコンパイル
pattern = re.compile(r'''
    ^       # 行頭
    (={2,}) # キャプチャ対象、2個以上の'='
    \s*     # 余分な0個以上の空白（'哲学'や'婚姻'の前後に余分な空白があるので除去）
    (.+?)   # キャプチャ対象、任意の文字が1文字以上、非貪欲（以降の条件の巻き込み防止）
    \s*     # 余分な0個以上の空白
    \1      # 後方参照、1番目のキャプチャ対象と同じ内容
    .*      # 任意の文字が0文字以上
    $       # 行末
    ''', re.MULTILINE + re.VERBOSE)

# 抽出
result = pattern.findall(extract_UK())

# 結果表示
for line in result:
    level = len(line[0]) - 1    # '='の数-1
    print('{indent}{sect}({level})'.format(
        indent='\t' * (level - 1), sect=line[1], level=level))
```

## 24. ファイル参照の抽出

記事から参照されているメディアファイルをすべて抜き出せ．

In [ ]:
print(getEnglandJson())

In [ ]:
import json
import re

pattern24 = re.compile(r'.*(?:File|ファイル):(.+?)\|.*', re.MULTILINE + re.VERBOSE)
result24 = pattern24.findall(getEnglandJson())

for line in result24:
    print(line)

#### qiitaの答え
```
# 正規表現のコンパイル
pattern = re.compile(r'''
    (?:File|ファイル)   # 非キャプチャ、'File'か'ファイル'
    :
    (.+?)   # キャプチャ対象、任意の文字1文字以上、非貪欲
    \|
    ''', re.VERBOSE)

# 抽出
result = pattern.findall(extract_UK())

# 結果表示
for line in result:
    print(line)
```

## 25. テンプレートの抽出

記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し，辞書オブジェクトとして格納せよ．

In [ ]:
print(getEnglandJson())

In [ ]:
import json
import re

pattern25 = re.compile(r'^.*(\{\{基礎情報.*\n\}\})$', re.DOTALL + re.MULTILINE)

kiso_joho = pattern25.findall(getEnglandJson())

#print(type(kiso_joho[0]))
#print(kiso_joho[0])

rows = kiso_joho[0].split("\n")
#print(rows)

kiso_dict = {}
#print(kiso_dict.keys)
#print(kiso_dict.items)

for row in rows:
    pattern = re.compile("\|(.*) = (.*)")
    result = pattern.findall(row)
    if len(result) > 0:
        kiso_dict[result[0][0]] = result[0][1]
    else:
        continue
print(kiso_dict)
for key in kiso_dict.keys():
    print(key)

#### qiitaの答え
```
# 基礎情報テンプレートの抽出条件のコンパイル
pattern = re.compile(r'''
    ^\{\{基礎情報.*?$   # '{{基礎情報'で始まる行
    (.*?)       # キャプチャ対象、任意の0文字以上、非貪欲
    ^\}\}$      # '}}'の行
    ''', re.MULTILINE + re.VERBOSE + re.DOTALL)

# 基礎情報テンプレートの抽出
contents = pattern.findall(extract_UK())

# 抽出結果からのフィールド名と値の抽出条件コンパイル
pattern = re.compile(r'''
    ^\|         # '|'で始まる行
    (.+?)       # キャプチャ対象（フィールド名）、任意の1文字以上、非貪欲
    \s*         # 空白文字0文字以上
    =
    \s*         # 空白文字0文字以上
    (.+?)       # キャプチャ対象（値）、任意の1文字以上、非貪欲
    (?:         # キャプチャ対象外のグループ開始
        (?=\n\|)    # 改行+'|'の手前（肯定の先読み）
        | (?=\n$)   # または、改行+終端の手前（肯定の先読み）
    )           # グループ終了
    ''', re.MULTILINE + re.VERBOSE + re.DOTALL)

# フィールド名と値の抽出
fields = pattern.findall(contents[0])

# 辞書にセット
result = {}
keys_test = []      # 確認用の出現順フィールド名リスト
for field in fields:
    result[field[0]] = field[1]
    keys_test.append(field[0])

# 確認のため表示（確認しやすいようにkeys_testを使ってフィールド名の出現順にソート）
for item in sorted(result.items(),
        key=lambda field: keys_test.index(field[0])):
    print(item)
```

## 26. 強調マークアップの除去

25の処理時に，テンプレートの値からMediaWikiの強調マークアップ（弱い強調，強調，強い強調のすべて）を除去してテキストに変換せよ（参考: [マークアップ早見表](http://ja.wikipedia.org/wiki/Help:%E6%97%A9%E8%A6%8B%E8%A1%A8)）．

#### qiitaの答え
```
# coding: utf-8

fname = 'hightemp.txt'
count = 0
with open(fname) as data_file:
    for line in data_file:
        count += 1
print(count)
```

## 27. 内部リンクの除去

26の処理に加えて，テンプレートの値からMediaWikiの内部リンクマークアップを除去し，テキストに変換せよ（参考: [マークアップ早見表](http://ja.wikipedia.org/wiki/Help:%E6%97%A9%E8%A6%8B%E8%A1%A8)）．

#### qiitaの答え
```
# coding: utf-8

fname = 'hightemp.txt'
count = 0
with open(fname) as data_file:
    for line in data_file:
        count += 1
print(count)
```

## 28. MediaWikiマークアップの除去

27の処理に加えて，テンプレートの値からMediaWikiマークアップを可能な限り除去し，国の基本情報を整形せよ．

#### qiitaの答え
```
# coding: utf-8

fname = 'hightemp.txt'
count = 0
with open(fname) as data_file:
    for line in data_file:
        count += 1
print(count)
```

## 29. 国旗画像のURLを取得する

テンプレートの内容を利用し，国旗画像のURLを取得せよ．（ヒント: [MediaWiki](http://www.mediawiki.org/wiki/API:Main_page/ja) APIの[imageinfo](http://www.mediawiki.org/wiki/API:Properties/ja#imageinfo_.2F_ii)を呼び出して，ファイル参照をURLに変換すればよい）

#### qiitaの答え
```
# coding: utf-8

fname = 'hightemp.txt'
count = 0
with open(fname) as data_file:
    for line in data_file:
        count += 1
print(count)
```